In [0]:
%pip install langchain langchain_community langchain_chroma

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.2/751.2 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.8/384.8 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 36.9 MB/s eta 0:00:00
     ━━━━━━

In [0]:
dbutils.library.restartPython()

# Loading data

In [0]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


- The webbaseloader loads the html using urllib
- bs_kwargs the beautiful soup is passed to parse the html page

In [0]:
bs4_strainer=bs4.SoupStrainer(class_=("post-title", "post-header","post-content"))
loader=WebBaseLoader(
  web_path=("https://lilianweng.github.io/posts/2023-06-23-agent/"),
  bs_kwargs={"parse_only": bs4_strainer}
)
docs=loader.load()
print(docs[0].page_content)
print(len(docs[0].page_content))



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:

Planning

Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.
Reflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.


Memory

Short-term memory: I 

# Split into chunks

> The document chunk size is 43k which is too big to fit in the context window of any LLM model and even if it fits the model's context window the model struggles to find information from the text
> The document is split using the RecursiveCharactertextsplitter as this recurssively tries to split the document using the common delimiters until each chunk is appropriate size.
> Here I am splitting the document into 800 character chunks and 100 character overlap. This overlap helps to mitigate the problem of separating a statement from importantn context related to it.

> The add_start_index adds the starting index of the other chunk in the metadata.


In [0]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100, add_start_index=True)
all_splits=text_splitter.split_documents(docs)
len(all_splits)

81

#  Store the chunks in a vector database

In [0]:
from openai import OpenAI
import os

# How to get your Databricks token: https://docs.databricks.com/en/dev-tools/auth/pat.html
DATABRICKS_TOKEN = os.environ.get('api_key')
# Alternatively in a Databricks notebook you can use this:
# DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

client = OpenAI(
  api_key=DATABRICKS_TOKEN,
  base_url="https://edp-prod-adl.cloud.databricks.com/serving-endpoints"
)

embeddings = client.embeddings.create(
  input='Your string for the embedding model goes here',
  model="databricks-gte-large-en"
)

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-2334371097182831>, line 1
----> 1 from openai import OpenAI
      2 import os
      4 # How to get your Databricks token: https://docs.databricks.com/en/dev-tools/auth/pat.html

ModuleNotFoundError: No module named 'openai'

In [0]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-2334371097182831>, line 1
----> 1 from openai import OpenAI
      2 import os
      4 # How to get your Databricks token: https://docs.databricks.com/en/dev-tools/auth/pat.html

ModuleNotFoundError: No module named 'openai'